## Fully Connected layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

# input dimensions
t_max = 2**8
dims = (1,1,4,1)
n_outputs = 10
np.random.seed(seed=35)

# model parameters
weights = (np.random.rand(n_outputs,np.product(dims[1:])) - 0.5) / 3 # weights = np.ones((n_outputs,np.product(dims[1:]))) # 
biases = (np.random.rand(n_outputs) - 0.5) / 2 # biases = np.zeros((n_outputs))# np.ones((n_outputs)) / 2 # 
inputs = np.random.rand(*dims) / 3 # np.ones((dims)) * 0.5 #  #values = np.zeros((dims)) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=dims[1:]),
    layers.Dense(weights=weights, biases=biases, rectifying=True, weight_acc=255),
])
# optional neuron probes
relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks[1::2]]

# build equivalent pyTorch model
pt_model = nn.Sequential(
    nn.Linear(in_features=np.product(dims[1:]), out_features=n_outputs), 
    nn.ReLU())
q_weights, q_biases, q_inputs = quartz.utils.quantize_values(weights, biases, inputs, loihi_model.layers[1].weight_acc, t_max)
pt_model[0].weight = torch.nn.Parameter(torch.tensor(q_weights))
pt_model[0].bias = torch.nn.Parameter(torch.tensor((q_biases)))
model_output = pt_model(torch.tensor(q_inputs.reshape(dims[0], -1))).detach().numpy()
model_output

In [ ]:
loihi_model(inputs, t_max)

In [ ]:
ok = [relco.plot() for relco in relcos]

## Convolutional Layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

t_max = 2**8
input_dims = (1,2,5,5) # (50,10,5,5) # 
weight_dims = (6,2,5,5) # (5,10,5,5) # 
padding = 0
groups = 1
kernel_size = weight_dims[2:]
np.random.seed(seed=27)
weights = (np.random.rand(*weight_dims)-0.5) / 3 # np.ones((weight_dims)) # 
biases = (np.random.rand(weight_dims[0])-0.5) / 2 # np.zeros((weight_dims[0])) # 
inputs = np.random.rand(*input_dims) / 3 # np.zeros((np.product(input_dims)))

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.Conv2D(weights=weights, biases=biases, padding=padding, groups=groups),
])

relcos = [quartz.probe(block) for block in loihi_model.layers[1].blocks]

q_weights, q_biases, q_inputs = quartz.utils.quantize_values(weights, biases, inputs, loihi_model.layers[1].weight_acc, t_max)
model = nn.Sequential(nn.Conv2d(in_channels=weight_dims[1]*groups, out_channels=weight_dims[0], kernel_size=kernel_size, padding=padding, groups=groups), nn.ReLU())
model[0].weight = torch.nn.Parameter(torch.tensor(q_weights))
model[0].bias = torch.nn.Parameter(torch.tensor((q_biases)))
model_output = model(torch.tensor(q_inputs)).detach().numpy()
model_output[0].max()

In [ ]:
loihi_output = loihi_model(inputs, t_max)

In [ ]:
ok = [relco.plot() for relco in relcos]

## Convpool layer

In [ ]:
import quartz
from quartz import layers
import numpy as np
import torch
import torch.nn as nn

input_dims = (1,2,4,4)
weight_dims = (5,2,3,3)
t_max = 2**8
conv_kernel_size = weight_dims[2:]
pooling_kernel_size = [2,2]
pooling_stride = 2

np.random.seed(seed=46)
np.set_printoptions(suppress=True)
weights = (np.random.rand(*weight_dims)-0.5) / 5 # np.zeros(weight_dims) #
biases = (np.random.rand(weight_dims[0])-0.5) # np.zeros(weight_dims[0]) # 

loihi_model = quartz.Network([
    layers.InputLayer(dims=input_dims[1:]),
    layers.ConvPool2D(weights=weights, biases=biases, pool_kernel_size=pooling_kernel_size),
    layers.MonitorLayer(),
])
trigger0 = quartz.probe(loihi_model.layers[0].blocks[0])
trigger1 = quartz.probe(loihi_model.layers[1].blocks[0])
trigger2 = quartz.probe(loihi_model.layers[2].blocks[0])
convmaxs1 = [quartz.probe(block) for block in loihi_model.layers[1].blocks[11:]]

values = np.random.rand(*input_dims) / 4
quantized_values = (values*t_max).round()/t_max
weight_acc = loihi_model.layers[1].weight_acc
quantized_weights = (weight_acc*weights).round()/weight_acc
quantized_biases = (biases*t_max).round()/t_max

model = nn.Sequential(
    nn.Conv2d(in_channels=weight_dims[1], out_channels=weight_dims[0], kernel_size=conv_kernel_size), nn.ReLU(),
    nn.MaxPool2d(kernel_size=pooling_kernel_size, stride=pooling_stride), nn.ReLU(),
)
model[0].weight = nn.Parameter(torch.tensor(quantized_weights))
model[0].bias = nn.Parameter(torch.tensor(quantized_biases))
model_output = model(torch.tensor(values)).detach().numpy()
model_output

In [ ]:
loihi_output = loihi_model(values, t_max)
loihi_output

In [ ]:
convmaxs1[0].output()


In [ ]:
trigger1.output()

In [ ]:
trigger2.output()

In [ ]:
loihi_model.data

In [ ]:
ok = [convmax.plot() for convmax in convmaxs1]